# Conformal changepoint localization (CONCH) experiments

This notebook implements the CONCH algorithm for changepoint localization. We apply our algorithm to image data from MNIST, text data from SST-2, and a Gaussian mean change.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import ks_2samp, ks_1samp, uniform
import torch
from torchvision.datasets import MNIST
from torchvision import transforms
from torchvision.models import ResNet18_Weights
from tqdm import tqdm

In [ ]:
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats(
    "svg"
)
plt.style.use("math.mplstyle")

plt.rcParams.update({"axes.labelsize": 14})
plt.rcParams.update({"xtick.labelsize": 14})
plt.rcParams.update({"ytick.labelsize": 14})
plt.rcParams.update({"legend.fontsize": 14})
plt.rcParams.update({"axes.titlesize": 16})

## MNIST simulation

CONCH algorithm applied to an image dataset with digit 3 before the changepoint and digit 7 after.

In [ ]:
def get_pretrained_model(device="cpu"):
    model = torch.hub.load(
        "pytorch/vision:v0.10.0", "resnet18", weights=ResNet18_Weights.IMAGENET1K_V1
    )

    model.conv1 = torch.nn.Conv2d(
        1, 64, kernel_size=7, stride=2, padding=3, bias=False
    )

    model.fc = torch.nn.Linear(model.fc.in_features, 10)
    model.to(device)
    model.eval()
    return model


def get_mnist_trained_model(device="cpu"):
    class MNISTModel(torch.nn.Module):
        def __init__(self):
            super(MNISTModel, self).__init__()
            self.conv1 = torch.nn.Conv2d(1, 32, 3, 1)
            self.conv2 = torch.nn.Conv2d(32, 64, 3, 1)
            self.dropout1 = torch.nn.Dropout(0.25)
            self.dropout2 = torch.nn.Dropout(0.5)
            self.fc1 = torch.nn.Linear(9216, 128)
            self.fc2 = torch.nn.Linear(128, 10)

        def forward(self, x):
            x = self.conv1(x)
            x = torch.nn.functional.relu(x)
            x = self.conv2(x)
            x = torch.nn.functional.relu(x)
            x = torch.nn.functional.max_pool2d(x, 2)
            x = self.dropout1(x)
            x = torch.flatten(x, 1)
            x = self.fc1(x)
            x = torch.nn.functional.relu(x)
            x = self.dropout2(x)
            x = self.fc2(x)
            return x

    model = MNISTModel().to(device)

    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )

    train_dataset = MNIST(root="./data", train=True, download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)

    optimizer = torch.optim.Adam(model.parameters())
    criterion = torch.nn.CrossEntropyLoss()

    print("Training MNIST model...")
    model.train()
    for epoch in range(1):
        for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(
                    f"Epoch: {epoch} [{batch_idx*len(data)}/{len(train_loader.dataset)} "
                    f"({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}"
                )

    test_dataset = MNIST(root="./data", train=False, download=True, transform=transform)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000)

    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = 100.0 * correct / len(test_loader.dataset)
    print(f"Test accuracy: {accuracy:.2f}%")

    model.eval()
    return model


def predict_digit(model, image, device="cpu"):
    image = image.reshape(1, 1, 28, 28)
    image_tensor = torch.tensor(image, device=device)

    with torch.no_grad():
        outputs = torch.softmax(model(image_tensor), dim=1).cpu()
        predicted = outputs.argmax(dim=1).item()
    return (predicted, outputs)

In [ ]:
def generate_mnist_dataset(length, changepoint, digit1=3, digit2=7):
    transform = transforms.ToTensor()
    mnist_data = MNIST(
        root="./data", train=True, download=True, transform=transform
    )
    data = mnist_data.data.numpy()
    targets = mnist_data.targets.numpy()
    
    images_digit1 = data[targets == digit1]
    images_digit2 = data[targets == digit2]
    np.random.shuffle(images_digit1)
    np.random.shuffle(images_digit2)
    
    n1 = changepoint + 1
    n2 = length - n1
    if n1 > len(images_digit1) or n2 > len(images_digit2):
        raise ValueError("Insufficient images for the specified digits and length.")
        
    data1 = images_digit1[:n1]
    data2 = images_digit2[:n2]
    x = np.concatenate([data1, data2], axis=0)
    
    x = x.reshape(length, -1).astype(np.float32) / 255.0
    return x

In [ ]:
def get_discrepancy_scores(x, scores_left, scores_right):
    discrepancy_scores = np.empty(len(x) - 1)
    statistics = []
    for t in tqdm(range(len(x) - 1)):
        p = np.empty(len(x))
        for r in range(t + 1):
            p[r] = (
                np.count_nonzero(scores_left[t, : r + 1] > scores_left[t, r])
                + np.random.uniform(0, 1) * np.count_nonzero(scores_left[t, : r + 1] == scores_left[t, r])
            ) / (r + 1)
        for r in range(len(x) - 1, t, -1):
            p[r] = (
                np.count_nonzero(scores_right[t, r:] > scores_right[t, r])
                + np.random.uniform(0, 1) * np.count_nonzero(scores_right[t, r:] == scores_right[t, r])
            ) / (len(x) - r)
        statistics.append((ks_1samp(p[: t + 1], uniform.cdf), ks_1samp(p[t+1:], uniform.cdf)))
        discrepancy_scores[t] = (
            statistics[-1][0].statistic * np.sqrt(t + 1)
            + statistics[-1][1].statistic * np.sqrt(len(x) - t - 1)
        )
    return discrepancy_scores, statistics

### Experiment parameters

In [ ]:
length = 1000
changepoint = 400
digit1 = 3
digit2 = 7

### Visualize the dataset

In [ ]:
x = generate_mnist_dataset(length, changepoint, digit1, digit2)

fig, axes = plt.subplots(1, 5, figsize=(6, 5))

for i, ax in enumerate(axes):
    idx = changepoint - 2 + i
    img = x[idx].reshape(28, 28)
    ax.imshow(img, cmap="gray")
    if i != 2:
        ax.set_title(f"$t = {idx}$")
    else:
        ax.set_title(f"$t = \\xi = {idx}$")
    ax.axis("off")

plt.tight_layout()
plt.savefig("images/mnist-sample.pdf")
plt.show()

### Digit classifier model

In [ ]:
model = get_mnist_trained_model()

In [ ]:
x = generate_mnist_dataset(length, changepoint, digit1, digit2)

predicted_digits = [predict_digit(model, x[i]) for i in tqdm(range(length))]
probabilities = torch.vstack([prob for _, prob in predicted_digits])

left_score = np.zeros((length, length))
seen_digits = {}
for t, (predicted, _) in enumerate(predicted_digits):
    if predicted in seen_digits:
        seen_digits[predicted] += 1
    else:
        seen_digits[predicted] = 1
    curr_digit = max(seen_digits, key=seen_digits.get)
    left_score[t, : t + 1] = probabilities[: t + 1, curr_digit].cpu() / (
        1 - probabilities[: t + 1, curr_digit].cpu()
    )

right_score = np.zeros((length, length))
seen_digits = {}
for i, (predicted, _) in enumerate(reversed(predicted_digits)):
    t = length - i - 1
    if predicted in seen_digits:
        seen_digits[predicted] += 1
    else:
        seen_digits[predicted] = 1
    curr_digit = max(seen_digits, key=seen_digits.get)
    right_score[t, t:] = probabilities[t:, curr_digit].cpu() / (
        1 - probabilities[t:, curr_digit].cpu()
    )

discrepancy_scores, statistics = get_discrepancy_scores(x, left_score, right_score)

In [ ]:
from scipy.stats import chi2

p_values_left = np.array([s[0].pvalue for s in statistics])
p_values_right = np.array([s[1].pvalue for s in statistics])

min_method = chi2.cdf(np.minimum(2 * p_values_left, 2 * p_values_right, np.ones_like(p_values_left)), 4)
threshold = 0.05

plt.plot(np.arange(1, length), min_method)
plt.axvline(
    changepoint, color="red", linestyle="--", label="Changepoint ($\\xi = 400$)"
)
plt.axhline(threshold, color='green', linestyle=':', label='Threshold ($\\alpha = 0.05$)')
plt.xlabel("$t$")
plt.ylabel("p-value ($p_t$)")
plt.title("p-values for MNIST digit change (digit classifier)")
plt.legend()
plt.savefig("images/mnist-pvalues.pdf")
plt.show()

In [ ]:
confidence_set = np.argwhere(min_method > threshold).flatten()
confidence_interval = (confidence_set[0], confidence_set[-1]) if len(confidence_set) > 0 else None

print(f"True changepoint: {changepoint}")
print(f"Confidence interval: {confidence_interval}")
print(f"Maximum p-value at t={np.argmax(min_method)+1}")

### Pre-trained ResNet-18 model

In [ ]:
pretrained_model = get_pretrained_model()

In [ ]:
length = 1000
changepoint = 400
digit1 = 3
digit2 = 7

x = generate_mnist_dataset(length, changepoint, digit1, digit2)

predicted_digits = [predict_digit(pretrained_model, x[i]) for i in tqdm(range(length))]
probabilities = torch.vstack([prob for _, prob in predicted_digits])

left_score = np.zeros((length, length))
seen_digits = {}
for t, (predicted, _) in enumerate(predicted_digits):
    if predicted in seen_digits:
        seen_digits[predicted] += 1
    else:
        seen_digits[predicted] = 1
    curr_digit = max(seen_digits, key=seen_digits.get)
    left_score[t, : t + 1] = probabilities[: t + 1, curr_digit].cpu() / (
        1 - probabilities[: t + 1, curr_digit].cpu()
    )

right_score = np.zeros((length, length))
seen_digits = {}
for i, (predicted, _) in enumerate(reversed(predicted_digits)):
    t = length - i - 1
    if predicted in seen_digits:
        seen_digits[predicted] += 1
    else:
        seen_digits[predicted] = 1
    curr_digit = max(seen_digits, key=seen_digits.get)
    right_score[t, t:] = probabilities[t:, curr_digit].cpu() / (
        1 - probabilities[t:, curr_digit].cpu()
    )

discrepancy_scores, statistics = get_discrepancy_scores(x, left_score, right_score)

In [ ]:
from scipy.stats import chi2

p_values_left = np.array([s[0].pvalue for s in statistics])
p_values_right = np.array([s[1].pvalue for s in statistics])

min_method_pretrained = chi2.cdf(np.minimum(2 * p_values_left, 2 * p_values_right, np.ones_like(p_values_left)), 4)
threshold = 0.05

plt.plot(np.arange(1, length), min_method_pretrained)
plt.axvline(
    changepoint, color="red", linestyle="--", label="Changepoint ($\\xi = 400$)"
)
plt.axhline(threshold, color='green', linestyle=':', label='Threshold ($\\alpha = 0.05$)')
plt.xlabel("$t$")
plt.ylabel("p-value ($p_t$)")
plt.title("p-values for MNIST digit change (pre-trained classifier)")
plt.legend()
plt.savefig("images/mnist-pretrained.pdf")
plt.show()

In [ ]:
confidence_set = np.argwhere(min_method_pretrained > threshold).flatten()
confidence_interval = (confidence_set[0], confidence_set[-1]) if len(confidence_set) > 0 else None

print(f"True changepoint: {changepoint}")
print(f"Confidence interval: {confidence_interval}")
print(f"Maximum p-value at t={np.argmax(min_method)+1}")

### Two-sided calibration with certified data

In [ ]:
def generate_extended_mnist_dataset(
    length, changepoint, calibration_size=100, digit1=3, digit2=7
):
    """Generate MNIST dataset with a changepoint and calibration data."""
    transform = transforms.ToTensor()
    mnist_data = MNIST(root="./data", train=True, download=True, transform=transform)
    data = mnist_data.data.numpy()
    targets = mnist_data.targets.numpy()

    images_digit1 = data[targets == digit1]
    images_digit2 = data[targets == digit2]
    np.random.shuffle(images_digit1)
    np.random.shuffle(images_digit2)

    n1 = changepoint + 1 + calibration_size
    n2 = (length - changepoint - 1) + calibration_size

    if n1 > len(images_digit1) or n2 > len(images_digit2):
        raise ValueError("Insufficient images for the specified digits and length.")

    data1 = images_digit1[:n1]
    data2 = images_digit2[:n2]

    calibration_pre = data1[:calibration_size]
    main_pre = data1[calibration_size:n1]
    calibration_post = data2[:calibration_size]
    main_post = data2[calibration_size:n2]

    x_main = np.concatenate([main_pre, main_post], axis=0)
    x_calibration_pre = calibration_pre
    x_calibration_post = calibration_post

    x_main = x_main.reshape(length, -1).astype(np.float32) / 255.0
    x_calibration_pre = (
        x_calibration_pre.reshape(calibration_size, -1).astype(np.float32) / 255.0
    )
    x_calibration_post = (
        x_calibration_post.reshape(calibration_size, -1).astype(np.float32) / 255.0
    )

    return x_main, x_calibration_pre, x_calibration_post

In [ ]:
calibration_size = 100

def compute_left_scores_with_calibration(
    probabilities, predicted_digits, probabilities_cal_pre, predicted_cal_pre, length
):
    left_scores = np.zeros((length, length))
    left_scores_cal = np.zeros((length, calibration_size))

    seen_digits = {}

    for i, (predicted, _) in enumerate(predicted_cal_pre):
        if predicted in seen_digits:
            seen_digits[predicted] += 1
        else:
            seen_digits[predicted] = 1

    for t, (predicted, _) in enumerate(predicted_digits):
        if predicted in seen_digits:
            seen_digits[predicted] += 1
        else:
            seen_digits[predicted] = 1
        curr_digit = max(seen_digits, key=seen_digits.get)

        left_scores[t, : t + 1] = probabilities[: t + 1, curr_digit].cpu() / (
            1 - probabilities[: t + 1, curr_digit].cpu()
        )

        left_scores_cal[t, :] = probabilities_cal_pre[:, curr_digit].cpu() / (
            1 - probabilities_cal_pre[:, curr_digit].cpu()
        )

    return left_scores, left_scores_cal


def compute_right_scores_with_calibration(
    probabilities, predicted_digits, probabilities_cal_post, predicted_cal_post, length
):
    calibration_size = len(
        predicted_cal_post
    )
    right_scores = np.zeros((length, length))
    right_scores_cal = np.zeros((length, calibration_size))

    seen_digits = {}

    for i, (predicted, _) in enumerate(predicted_cal_post):
        if predicted in seen_digits:
            seen_digits[predicted] += 1
        else:
            seen_digits[predicted] = 1

    for i, (predicted, _) in enumerate(reversed(predicted_digits)):
        t = length - i - 1
        if predicted in seen_digits:
            seen_digits[predicted] += 1
        else:
            seen_digits[predicted] = 1
        curr_digit = max(seen_digits, key=seen_digits.get)

        right_scores[t, t:] = probabilities[t:, curr_digit].cpu() / (
            1 - probabilities[t:, curr_digit].cpu()
        )

        right_scores_cal[t, :] = probabilities_cal_post[:, curr_digit].cpu() / (
            1 - probabilities_cal_post[:, curr_digit].cpu()
        )

    return right_scores, right_scores_cal


def get_discrepancy_scores_with_calibration(
    x, scores_left, scores_right, scores_left_cal, scores_right_cal
):
    n = len(x)
    calibration_size_pre = scores_left_cal.shape[1]
    calibration_size_post = scores_right_cal.shape[1]
    discrepancy_scores = np.empty(n - 1)
    statistics = []

    for t in tqdm(range(n - 1)):
        p = np.empty(n)

        for r in range(t + 1):
            score_r = scores_left[t, r]

            main_counts = np.sum(scores_left[t, : r + 1] < score_r) + np.random.uniform(
                0, 1
            ) * np.sum(scores_left[t, : r + 1] == score_r)

            cal_counts = np.sum(scores_left_cal[t, :] < score_r) + np.random.uniform(
                0, 1
            ) * np.sum(scores_left_cal[t, :] == score_r)

            p[r] = (main_counts + cal_counts) / (r + 1 + calibration_size_pre)

        for r in range(n - 1, t, -1):
            score_r = scores_right[t, r]

            main_counts = np.sum(scores_right[t, r:] < score_r) + np.random.uniform(
                0, 1
            ) * np.sum(scores_right[t, r:] == score_r)

            cal_counts = np.sum(scores_right_cal[t, :] < score_r) + np.random.uniform(
                0, 1
            ) * np.sum(scores_right_cal[t, :] == score_r)

            p[r] = (main_counts + cal_counts) / (n - r + calibration_size_post)

        statistics.append(
            (ks_1samp(p[: t + 1], uniform.cdf), ks_1samp(p[t + 1 :], uniform.cdf))
        )
        discrepancy_scores[t] = statistics[-1][0].statistic * np.sqrt(
            t + 1
        ) + statistics[-1][1].statistic * np.sqrt(n - t - 1)

    return discrepancy_scores, statistics

In [ ]:
x_main, x_cal_pre, x_cal_post = generate_extended_mnist_dataset(
    length, changepoint, calibration_size, digit1, digit2
)

predicted_digits = [predict_digit(model, x_main[i]) for i in tqdm(range(length))]
probabilities = torch.vstack([prob for _, prob in predicted_digits])

predicted_cal_pre = [
    predict_digit(model, x_cal_pre[i]) for i in tqdm(range(calibration_size))
]
probabilities_cal_pre = torch.vstack([prob for _, prob in predicted_cal_pre])

predicted_cal_post = [
    predict_digit(model, x_cal_post[i]) for i in tqdm(range(calibration_size))
]
probabilities_cal_post = torch.vstack([prob for _, prob in predicted_cal_post])

left_scores, left_scores_cal = compute_left_scores_with_calibration(
    probabilities, predicted_digits, probabilities_cal_pre, predicted_cal_pre, length
)

right_scores, right_scores_cal = compute_right_scores_with_calibration(
    probabilities, predicted_digits, probabilities_cal_post, predicted_cal_post, length
)

discrepancy_scores_cal, statistics_cal = get_discrepancy_scores_with_calibration(
    x_main, left_scores, right_scores, left_scores_cal, right_scores_cal
)

In [ ]:
from scipy.stats import chi2

p_values_left = np.array([s[0].pvalue for s in statistics_cal])
p_values_right = np.array([s[1].pvalue for s in statistics_cal])

min_method = chi2.cdf(
    np.minimum(2 * p_values_left, 2 * p_values_right, np.ones_like(p_values_left)), 4
)
threshold = 0.05

plt.plot(np.arange(1, length), min_method)
plt.axvline(
    changepoint, color="red", linestyle="--", label="Changepoint ($\\xi = 400$)"
)
plt.axhline(
    threshold, color="green", linestyle=":", label="Threshold ($\\alpha = 0.05$)"
)
plt.xlabel("$t$")
plt.ylabel("p-value ($p_t$)")
plt.title("p-values for MNIST digit change (two-sided calibration)")
plt.legend()
plt.savefig("images/mnist-pvalues-calibrated.pdf")
plt.show()

In [ ]:
ci = np.argwhere(min_method > threshold).flatten()
confidence_interval = (ci[0], ci[-1]) if len(ci) > 0 else None
print(f"True changepoint: {changepoint}")
print(f"Confidence interval: {confidence_interval}")
print(f"Maximum p-value at t={np.argmax(min_method)+1}")

### Left-sided calibration with certified data

In [ ]:
def generate_left_calibration_mnist_dataset(
    length, changepoint, calibration_size=100, digit1=3, digit2=7
):
    transform = transforms.ToTensor()
    mnist_data = MNIST(root="./data", train=True, download=True, transform=transform)
    data = mnist_data.data.numpy()
    targets = mnist_data.targets.numpy()

    images_digit1 = data[targets == digit1]
    images_digit2 = data[targets == digit2]
    np.random.shuffle(images_digit1)
    np.random.shuffle(images_digit2)

    n1 = changepoint + 1 + calibration_size
    n2 = length - changepoint - 1

    if n1 > len(images_digit1) or n2 > len(images_digit2):
        raise ValueError("Insufficient images for the specified digits and length.")

    data1 = images_digit1[:n1]
    data2 = images_digit2[:n2]

    calibration_pre = data1[:calibration_size]
    main_pre = data1[calibration_size:n1]
    main_post = data2

    x_main = np.concatenate([main_pre, main_post], axis=0)
    x_calibration_pre = calibration_pre

    x_main = x_main.reshape(length, -1).astype(np.float32) / 255.0
    x_calibration_pre = (
        x_calibration_pre.reshape(calibration_size, -1).astype(np.float32) / 255.0
    )

    return x_main, x_calibration_pre

In [ ]:
def compute_left_scores_with_left_calibration(
    probabilities, predicted_digits, probabilities_cal_pre, predicted_cal_pre, length
):
    left_scores = np.zeros((length, length))
    left_scores_cal = np.zeros((length, len(predicted_cal_pre)))

    seen_digits = {}

    for i, (predicted, _) in enumerate(predicted_cal_pre):
        if predicted in seen_digits:
            seen_digits[predicted] += 1
        else:
            seen_digits[predicted] = 1

    for t, (predicted, _) in enumerate(predicted_digits):
        if predicted in seen_digits:
            seen_digits[predicted] += 1
        else:
            seen_digits[predicted] = 1
        curr_digit = max(seen_digits, key=seen_digits.get)

        left_scores[t, : t + 1] = probabilities[: t + 1, curr_digit].cpu() / (
            1 - probabilities[: t + 1, curr_digit].cpu()
        )

        left_scores_cal[t, :] = probabilities_cal_pre[:, curr_digit].cpu() / (
            1 - probabilities_cal_pre[:, curr_digit].cpu()
        )

    return left_scores, left_scores_cal


def compute_right_scores_without_calibration(probabilities, predicted_digits, length):
    right_scores = np.zeros((length, length))

    seen_digits = {}

    for i, (predicted, _) in enumerate(reversed(predicted_digits)):
        t = length - i - 1
        if predicted in seen_digits:
            seen_digits[predicted] += 1
        else:
            seen_digits[predicted] = 1
        curr_digit = max(seen_digits, key=seen_digits.get)

        right_scores[t, t:] = probabilities[t:, curr_digit].cpu() / (
            1 - probabilities[t:, curr_digit].cpu()
        )

    return right_scores

In [ ]:
def get_discrepancy_scores_with_left_calibration(
    x, scores_left, scores_right, scores_left_cal
):
    n = len(x)
    calibration_size_pre = scores_left_cal.shape[1]
    discrepancy_scores = np.empty(n - 1)
    statistics = []

    for t in tqdm(range(n - 1)):
        p = np.empty(n)

        for r in range(t + 1):
            score_r = scores_left[t, r]

            main_counts = np.sum(scores_left[t, : r + 1] < score_r) + np.random.uniform(
                0, 1
            ) * np.sum(scores_left[t, : r + 1] == score_r)

            cal_counts = np.sum(scores_left_cal[t, :] < score_r) + np.random.uniform(
                0, 1
            ) * np.sum(scores_left_cal[t, :] == score_r)

            p[r] = (main_counts + cal_counts) / (r + 1 + calibration_size_pre)

        for r in range(n - 1, t, -1):
            p[r] = (
                np.count_nonzero(scores_right[t, r:] > scores_right[t, r])
                + np.random.uniform(0, 1)
                * np.count_nonzero(scores_right[t, r:] == scores_right[t, r])
            ) / (n - r)

        statistics.append(
            (ks_1samp(p[: t + 1], uniform.cdf), ks_1samp(p[t + 1 :], uniform.cdf))
        )
        discrepancy_scores[t] = statistics[-1][0].statistic * np.sqrt(
            t + 1
        ) + statistics[-1][1].statistic * np.sqrt(n - t - 1)

    return discrepancy_scores, statistics

In [ ]:
length = 1000
changepoint = 400
calibration_size = 100
digit1 = 3
digit2 = 7

x_main, x_cal_pre = generate_left_calibration_mnist_dataset(
    length, changepoint, calibration_size, digit1, digit2
)

predicted_digits = [predict_digit(model, x_main[i]) for i in tqdm(range(length))]
probabilities = torch.vstack([prob for _, prob in predicted_digits])

predicted_cal_pre = [
    predict_digit(model, x_cal_pre[i]) for i in tqdm(range(calibration_size))
]
probabilities_cal_pre = torch.vstack([prob for _, prob in predicted_cal_pre])

left_scores, left_scores_cal = compute_left_scores_with_left_calibration(
    probabilities, predicted_digits, probabilities_cal_pre, predicted_cal_pre, length
)

right_scores = compute_right_scores_without_calibration(
    probabilities, predicted_digits, length
)

discrepancy_scores_left_cal, statistics_left_cal = (
    get_discrepancy_scores_with_left_calibration(
        x_main, left_scores, right_scores, left_scores_cal
    )
)

In [ ]:
p_values_left = np.array([s[0].pvalue for s in statistics_left_cal])
p_values_right = np.array([s[1].pvalue for s in statistics_left_cal])

min_method = chi2.cdf(
    np.minimum(2 * p_values_left, 2 * p_values_right, np.ones_like(p_values_left)), 4
)
threshold = 0.05

plt.plot(np.arange(1, length), min_method)
plt.axhline(
    threshold, color="green", linestyle=":", label="Threshold ($\\alpha = 0.05$)"
)
plt.axvline(
    changepoint, color="red", linestyle="--", label="Changepoint ($\\xi = 400$)"
)
plt.xlabel("$t$")
plt.ylabel("p-value ($p_t$)")
plt.title("p-values for MNIST digit change (left-side calibration)")
plt.legend()
plt.savefig("images/mnist-pvalues-left-calibrated.pdf")
plt.show()

In [ ]:
confidence_set = np.argwhere(min_method > threshold).flatten()
confidence_interval = (
    (confidence_set[0], confidence_set[-1]) if len(confidence_set) > 0 else None
)

print(f"True changepoint: {changepoint}")
print(f"Confidence interval: {confidence_interval}")
print(f"Maximum p-value at t={np.argmax(min_method)+1}")

## SST-2 simulation

CONCH algorithm applied to a text dataset with positive movie reviews before the changepoint and negative movie reviews after.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
from scipy.stats import ks_2samp, ks_1samp, uniform
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset

import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
plt.style.use("math.mplstyle")

In [ ]:
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
plt.style.use("math.mplstyle")

In [ ]:
def get_pretrained_sentiment_model(device="cpu"):
    model_name = "distilbert-base-uncased-finetuned-sst-2-english"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    model.to(device)
    model.eval()
    return model, tokenizer

In [ ]:
def generate_sentiment_dataset(length, changepoint, dataset_name="sst2"):
    dataset = load_dataset(dataset_name)

    train_data = dataset["train"]
    positive_texts = [item["sentence"] for item in train_data if item["label"] == 1]
    negative_texts = [item["sentence"] for item in train_data if item["label"] == 0]

    random.shuffle(positive_texts)
    random.shuffle(negative_texts)

    n1 = changepoint + 1
    n2 = length - n1

    if n1 > len(positive_texts) or n2 > len(negative_texts):
        raise ValueError("Insufficient texts for the specified length and changepoint.")

    texts_before = positive_texts[:n1]
    texts_after = negative_texts[:n2]

    texts = texts_before + texts_after
    true_labels = [1] * n1 + [0] * n2

    return texts, true_labels

In [ ]:
def predict_sentiment(model, tokenizer, text, device="cpu"):
    inputs = tokenizer(
        text, return_tensors="pt", padding=True, truncation=True, max_length=512
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1).cpu()
        predicted = probs.argmax(dim=1).item()

    return (predicted, probs.squeeze())

### Create dataset and see examples

In [ ]:
length = 1000
changepoint = 400
device = "cuda" if torch.cuda.is_available() else "cpu"

model, tokenizer = get_pretrained_sentiment_model(device)

print("Generating sentiment dataset...")
texts, true_labels = generate_sentiment_dataset(length, changepoint)

print("Getting predictions...")
predictions = []
probabilities = []

for i, text in enumerate(tqdm(texts)):
    pred, prob = predict_sentiment(model, tokenizer, text, device)
    predictions.append(pred)
    probabilities.append(prob)

probabilities = torch.stack(
    probabilities
)

In [ ]:
print("\nExamples before changepoint (positive):")
for i in range(3):
    idx = np.random.randint(0, changepoint)
    print(f'Text {i+1}: "{texts[idx]}"')
    print(
        f"True label: Positive, Predicted: {'Positive' if predictions[idx] == 1 else 'Negative'}"
    )
    print(f"Confidence: {probabilities[idx][predictions[idx]]:.4f}\n")

print("\nExamples after changepoint (negative):")
for i in range(3):
    idx = np.random.randint(changepoint + 1, length)
    print(f'Text {i+1}: "{texts[idx]}"')
    print(
        f"True label: Negative, Predicted: {'Positive' if predictions[idx] == 1 else 'Negative'}"
    )
    print(f"Confidence: {probabilities[idx][predictions[idx]]:.4f}\n")

### Full sentiment change (100% positive to 100% negative)

In [ ]:
left_score = np.zeros((length, length))
seen_sentiments = {0: 0, 1: 0}

for t in range(length):
    seen_sentiments[predictions[t]] += 1
    curr_sentiment = max(seen_sentiments, key=seen_sentiments.get)

    for r in range(t + 1):
        left_score[t, r] = probabilities[r, curr_sentiment] / (
            1 - probabilities[r, curr_sentiment]
        )

right_score = np.zeros((length, length))
seen_sentiments = {0: 0, 1: 0}

for i in range(length - 1, -1, -1):
    t = length - i - 1
    seen_sentiments[predictions[i]] += 1
    curr_sentiment = max(seen_sentiments, key=seen_sentiments.get)

    right_score[t, t:] = probabilities[t:, curr_sentiment] / (
        1 - probabilities[t:, curr_sentiment]
    )

In [ ]:
def get_discrepancy_scores(scores_left, scores_right, length):
    discrepancy_scores = np.empty(length - 1)
    statistics = []

    for t in tqdm(range(length - 1)):
        p = np.empty(length)

        for r in range(t + 1):
            p[r] = (
                np.count_nonzero(scores_left[t, : r + 1] > scores_left[t, r])
                + np.random.uniform(0, 1)
                * np.count_nonzero(scores_left[t, : r + 1] == scores_left[t, r])
            ) / (r + 1)

        for r in range(length - 1, t, -1):
            p[r] = (
                np.count_nonzero(scores_right[t, r:] > scores_right[t, r])
                + np.random.uniform(0, 1)
                * np.count_nonzero(scores_right[t, r:] == scores_right[t, r])
            ) / (length - r)

        statistics.append(
            (ks_1samp(p[: t + 1], uniform.cdf), ks_1samp(p[t + 1 :], uniform.cdf))
        )
        discrepancy_scores[t] = statistics[-1][0].statistic * np.sqrt(
            t + 1
        ) + statistics[-1][1].statistic * np.sqrt(length - t - 1)

    return discrepancy_scores, statistics

In [ ]:
discrepancy_scores, statistics = get_discrepancy_scores(left_score, right_score, length)

In [ ]:
from scipy.stats import chi2

p_values_left = np.array([s[0].pvalue for s in statistics])
p_values_right = np.array([s[1].pvalue for s in statistics])

min_method = chi2.cdf(
    np.minimum(2 * p_values_left, 2 * p_values_right, np.ones_like(p_values_left)), 4
)
threshold = 0.05

plt.plot(np.arange(1, length), min_method)
plt.axvline(
    changepoint,
    color="red",
    linestyle="--",
    label=f"Changepoint ($\\xi = {changepoint}$)",
)
plt.axhline(
    threshold,
    color="green",
    linestyle=":",
    label=f"Threshold ($\\alpha = {threshold}$)",
)
plt.xlabel("$t$")
plt.ylabel("p-value ($p_t$)")
plt.title("p-values for SST-2 sentiment change")
plt.legend()
plt.savefig("images/sentiment-pvalues.pdf")
plt.show()

confidence_set = np.argwhere(min_method > threshold).flatten()
confidence_interval = (
    (confidence_set[0], confidence_set[-1]) if len(confidence_set) > 0 else None
)

print(f"True changepoint: {changepoint}")
print(f"Confidence interval: {confidence_interval}")
print(f"Maximum p-value at t={np.argmax(min_method)+1}")

### Mixed sentiment change (60% positive to 60% negative)

In [ ]:
def generate_mixed_sentiment_dataset(length, changepoint, dataset_name="sst2"):
    dataset = load_dataset(dataset_name)

    train_data = dataset["train"]
    positive_texts = [item["sentence"] for item in train_data if item["label"] == 1]
    negative_texts = [item["sentence"] for item in train_data if item["label"] == 0]

    random.shuffle(positive_texts)
    random.shuffle(negative_texts)

    n_pre = changepoint + 1
    n_post = length - n_pre

    n_pos_pre = int(n_pre * 0.6)
    n_neg_pre = n_pre - n_pos_pre

    n_pos_post = int(n_post * 0.4)
    n_neg_post = n_post - n_pos_post

    if n_pos_pre + n_pos_post > len(positive_texts) or n_neg_pre + n_neg_post > len(
        negative_texts
    ):
        raise ValueError(
            "Insufficient texts for the specified distribution and length."
        )

    pre_pos_texts = positive_texts[:n_pos_pre]
    pre_neg_texts = negative_texts[:n_neg_pre]
    pre_texts = pre_pos_texts + pre_neg_texts
    pre_labels = [1] * n_pos_pre + [0] * n_neg_pre

    pre_combined = list(zip(pre_texts, pre_labels))
    random.shuffle(pre_combined)
    pre_texts, pre_labels = zip(*pre_combined)

    post_pos_texts = positive_texts[n_pos_pre : n_pos_pre + n_pos_post]
    post_neg_texts = negative_texts[n_neg_pre : n_neg_pre + n_neg_post]
    post_texts = post_pos_texts + post_neg_texts
    post_labels = [1] * n_pos_post + [0] * n_neg_post

    post_combined = list(zip(post_texts, post_labels))
    random.shuffle(post_combined)
    post_texts, post_labels = zip(*post_combined)

    texts = list(pre_texts) + list(post_texts)
    true_labels = list(pre_labels) + list(post_labels)

    return texts, true_labels

In [ ]:
length = 1000
changepoint = 400
device = "cuda" if torch.cuda.is_available() else "cpu"

if "model" not in locals() or "tokenizer" not in locals():
    model, tokenizer = get_pretrained_sentiment_model(device)

print("Generating mixed sentiment dataset (60%/40% to 40%/60%)...")
texts, true_labels = generate_mixed_sentiment_dataset(length, changepoint)

print("Getting predictions...")
predictions = []
probabilities = []

for i, text in enumerate(tqdm(texts)):
    pred, prob = predict_sentiment(model, tokenizer, text, device)
    predictions.append(pred)
    probabilities.append(prob)

probabilities = torch.stack(
    probabilities
)

left_score = np.zeros((length, length))
seen_sentiments = {0: 0, 1: 0}

for t in range(length):
    seen_sentiments[predictions[t]] += 1
    curr_sentiment = max(seen_sentiments, key=seen_sentiments.get)

    for r in range(t + 1):
        left_score[t, r] = probabilities[r, curr_sentiment] / (
            1 - probabilities[r, curr_sentiment]
        )

right_score = np.zeros((length, length))
seen_sentiments = {0: 0, 1: 0}

for i in range(length - 1, -1, -1):
    t = length - i - 1
    seen_sentiments[predictions[i]] += 1
    curr_sentiment = max(seen_sentiments, key=seen_sentiments.get)

    right_score[t, t:] = probabilities[t:, curr_sentiment] / (
        1 - probabilities[t:, curr_sentiment]
    )

discrepancy_scores, statistics = get_discrepancy_scores(left_score, right_score, length)

p_values_left = np.array([s[0].pvalue for s in statistics])
p_values_right = np.array([s[1].pvalue for s in statistics])

min_method = chi2.cdf(
    np.minimum(2 * p_values_left, 2 * p_values_right, np.ones_like(p_values_left)), 4
)
threshold = 0.05

In [ ]:
confidence_set = np.argwhere(min_method > threshold).flatten()
confidence_interval = (
    (confidence_set[0], confidence_set[-1]) if len(confidence_set) > 0 else None
)

print(f"True changepoint: {changepoint}")
print(f"Confidence interval: {confidence_interval}")
print(f"Maximum p-value at t={np.argmax(min_method)+1}")

In [ ]:
pre_positives = sum(1 for i in range(changepoint + 1) if true_labels[i] == 1)
pre_negatives = (changepoint + 1) - pre_positives
post_positives = sum(1 for i in range(changepoint + 1, length) if true_labels[i] == 1)
post_negatives = (length - changepoint - 1) - post_positives

print("\nSentiment distribution in data:")
print(
    f"Pre-change: {pre_positives/(changepoint+1)*100:.1f}% positive, {pre_negatives/(changepoint+1)*100:.1f}% negative"
)
print(
    f"Post-change: {post_positives/(length-changepoint-1)*100:.1f}% positive, {post_negatives/(length-changepoint-1)*100:.1f}% negative"
)

In [ ]:
np.argwhere(min_method > threshold).flatten()

In [ ]:
plt.plot(np.arange(1, length), min_method)
plt.axvline(
    changepoint,
    color="red",
    linestyle="--",
    label=f"Changepoint ($\\xi = {changepoint}$)",
)
plt.axhline(
    threshold,
    color="green",
    linestyle=":",
    label=f"Threshold ($\\alpha = {threshold}$)",
)
plt.xlabel("$t$")
plt.ylabel("p-value ($p_t$)")
plt.title(
    "p-values for SST-2 mixed sentiment change"
)
plt.legend()
plt.savefig("images/sentiment-pvalues-mixed.pdf")
plt.show()

## Gaussian mean change

CONCH algorithm applied to a numeric dataset with a Gaussian mean change at the changepoint.

In [ ]:
def plot_pvalue_matrix(
    pvalues, changepoint=None, title=None, save_path=None, log_transform=True
):
    if log_transform:
        plot_data = -np.log10(np.clip(pvalues, 1e-10, 1))
        label = "$-\\log_{10}(p_t)$"
    else:
        plot_data = pvalues
        label = "p-value"

    fig1 = plt.figure(figsize=(10, 7))
    ax1 = fig1.add_subplot(111)
    ax1.grid(False)
    ax1.invert_yaxis()

    im1 = ax1.imshow(plot_data, cmap="viridis", aspect="auto")

    cbar1 = fig1.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)
    cbar1.set_label(label, fontsize=18)
    cbar1.ax.tick_params(labelsize=16)

    ax1.set_xlabel("$r$", fontsize=18)
    ax1.set_ylabel("$t$", fontsize=18)
    ax1.tick_params(axis="both", which="major", labelsize=16)

    if title:
        ax1.set_title(title, fontsize=18)
    else:
        ax1.set_title("MCP", fontsize=18)

    fig2 = plt.figure(figsize=(10, 7))
    ax2 = fig2.add_subplot(111)
    ax2.grid(False)
    ax2.invert_yaxis()

    im2 = ax2.imshow(plot_data, cmap="viridis", aspect="auto")

    cbar2 = fig2.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)
    cbar2.set_label(label, fontsize=18)
    cbar2.ax.tick_params(labelsize=16)

    t_indices, r_indices = np.indices(pvalues.shape)

    condition1 = (t_indices <= 400) & (r_indices <= t_indices)
    condition2 = (t_indices >= 400) & (r_indices > t_indices)
    valid_mask = condition1 | condition2

    validity_overlay = np.zeros((*pvalues.shape, 4))  # RGBA

    validity_overlay[valid_mask] = [0, 1, 0, 0.3]

    validity_overlay[~valid_mask] = [1, 0, 0, 0.3]

    if pvalues.shape[0] > 400:
        validity_overlay[398:403, :] = [
            1,
            1,
            0,
            0.7,
        ]

    ax2.imshow(validity_overlay, aspect="auto")

    ax2.set_xlabel("$r$", fontsize=18)
    ax2.set_ylabel("$t$", fontsize=18)
    ax2.tick_params(axis="both", which="major", labelsize=16)
    ax2.set_title("Validity of conformal p-values", fontsize=18)

    from matplotlib.patches import Patch

    legend_elements = [
        Patch(facecolor="green", alpha=0.5, label="Valid p-values"),
        Patch(facecolor="red", alpha=0.5, label="Invalid p-values"),
        Patch(facecolor="yellow", alpha=0.7, label="All valid p-values ($t=400$)"),
    ]

    ax2.legend(handles=legend_elements, fontsize=18)

    fig1.tight_layout()
    fig2.tight_layout()
    fig1.savefig("images/mcp.pdf")
    fig2.savefig("images/mcp-validity.pdf")
    
    return fig1, fig2

### Oracle (LR) score function

In [ ]:
def get_dataset(length, changepoint, signal_strength):
    x = np.random.normal(-signal_strength, 1, length)
    x[changepoint + 1 :] += 2 * signal_strength
    return x


def oracle_score(z, delta):
    return np.exp(-((z - delta) ** 2) / 2) / np.exp(-((z + delta) ** 2) / 2)


plot_mcp = False

def get_discrepancy_scores(x, scores_left, scores_right):
    discrepancy_scores = np.empty(len(x) - 1)
    statistics = []
    if plot_mcp:
        all_pvalues = np.empty((len(x) - 1, len(x)))
    for t in range(len(x) - 1):
        p = np.empty(len(x))
        for r in range(t + 1):
            p[r] = (
                np.count_nonzero(scores_left[: r + 1] > scores_left[r])
                + np.random.uniform(0, 1)
                * np.count_nonzero(scores_left[: r + 1] == scores_left[r])
            ) / (r + 1)
        for r in range(len(x) - 1, t, -1):
            p[r] = (
                np.count_nonzero(scores_right[r:] < scores_right[r])
                + np.random.uniform(0, 1)
                * np.count_nonzero(scores_right[r:] == scores_right[r])
            ) / (len(x) - r)
        statistics.append(
            (
                ks_1samp(p[: t + 1], uniform.cdf, method="exact"),
                ks_1samp(p[t + 1 :], uniform.cdf, method="exact"),
            )
        )
        discrepancy_scores[t] = statistics[-1][0].statistic * np.sqrt(
            t + 1
        ) + statistics[-1][1].statistic * np.sqrt(len(x) - t)

        if plot_mcp:
            all_pvalues[t, :] = p
    
    if plot_mcp:
        plot_pvalue_matrix(all_pvalues, changepoint, "Matrix of conformal p-values (MCP)", "images/mcp.pdf")

    return discrepancy_scores, statistics


def get_log_likelihood(x, delta):
    log_likelihood = np.empty(len(x) - 1)
    for t in range(len(x) - 1):
        log_likelihood[t] = (
            -np.sum((x[: t + 1] + delta) ** 2) / 2
            - np.sum((x[t + 1 :] - delta) ** 2) / 2
            - len(x) / 2 * np.log(2 * np.pi)
        )
    return log_likelihood

In [ ]:
delta = 1
length = 1000
changepoint = 400
signal_strength = 1
x = get_dataset(length, changepoint, signal_strength)
plt.plot(x)

In [ ]:
scores_left = oracle_score(x, delta)
scores_right = -1 / scores_left
discrepancy_scores, statistics = get_discrepancy_scores(x, scores_left, scores_right)

pvalues_left = np.array([s[0].pvalue for s in statistics])
pvalues_right = np.array([s[1].pvalue for s in statistics])
min_method = chi2.cdf(
    np.minimum(2 * pvalues_left, 2 * pvalues_right, np.ones_like(pvalues_left)), 4
)
plt.figure()
plt.plot(np.arange(1, length), min_method)
plt.axvline(
    changepoint, color="red", linestyle="--", label=f"Changepoint ($\\xi = {changepoint}$)"
)
plt.axhline(
    0.05, color="green", linestyle=":", label=f"Threshold ($\\alpha = 0.05$)"
)
plt.xlabel("$t$")
plt.ylabel("p-value ($p_t$)")
plt.title("p-values for Gaussian mean change (oracle score)")
plt.legend()
plt.savefig("images/oracle.pdf")
plt.show()

In [ ]:
ci = np.argwhere(min_method > 0.05).flatten()
confidence_interval = (ci[0], ci[-1]) if len(ci) > 0 else None
print(f"True changepoint: {changepoint}")
print(f"Confidence interval: {confidence_interval}")
print(f"Maximum p-value at t={np.argmax(min_method)+1}")

### Learned score function

In [ ]:
def kde_score(x):
    from scipy.stats import norm, gaussian_kde, chi2, uniform, ks_1samp

    length = len(x)
    all_discrepancy_scores = []
    all_statistics = []

    def compute_kde(data):
        data = np.asarray(data).reshape(-1)
        if len(data) <= 1:
            mean = data[0] if len(data) == 1 else 0
            return lambda x: norm.pdf(x, loc=mean, scale=0.1)
        return gaussian_kde(data)

    for t in tqdm(range(length - 1)):
        kde_left = compute_kde(x[: t + 1])
        kde_right = compute_kde(x[t + 1 :])

        scores_left = np.empty(length)
        for i in range(length):
            p_left = kde_left(x[i])
            p_right = kde_right(x[i])
            scores_left[i] = p_right / (p_left + 1e-10)

        scores_right = -1 / (scores_left + 1e-10)

        disc_scores, stats = get_discrepancy_scores(x, scores_left, scores_right)
        all_discrepancy_scores.append(disc_scores[t])
        all_statistics.append(stats[t])

    all_discrepancy_scores = np.array(all_discrepancy_scores)

    pvalues_left = np.array([s[0].pvalue for s in all_statistics])
    pvalues_right = np.array([s[1].pvalue for s in all_statistics])
    bonferroni_method = np.minimum(
        np.minimum(2 * pvalues_left, 2 * pvalues_right), np.ones(len(pvalues_left))
    )

    return all_discrepancy_scores, bonferroni_method

discrepancy_scores, min_method = kde_score(x)
plt.plot(np.arange(1, length), min_method)
plt.axvline(
    changepoint, color="red", linestyle="--", label=f"Changepoint ($\\xi = {changepoint}$)"
)
plt.axhline(
    0.05, color="green", linestyle=":", label=f"Threshold ($\\alpha = 0.05$)"
)
plt.xlabel("$t$")
plt.ylabel("p-value ($p_t$)")
plt.title("p-values for Gaussian mean change (learned score)")
plt.legend()
plt.savefig("images/learned.pdf")
plt.show()

In [ ]:
confidence_set = np.argwhere(min_method > 0.05).flatten()
confidence_interval = (
    (confidence_set[0], confidence_set[-1]) if len(confidence_set) > 0 else None
)
print(f"True changepoint: {changepoint}")
print(f"Confidence interval: {confidence_interval}")
print(f"Maximum p-value at t={np.argmax(min_method)+1}")

plt.plot(np.arange(1, length), min_method)
plt.axvline(
    changepoint,
    color="red",
    linestyle="--",
    label=f"Changepoint ($\\xi = {changepoint}$)",
)
plt.axhline(0.05, color="green", linestyle=":", label=f"Threshold ($\\alpha = 0.05$)")
plt.xlabel("$t$")
plt.ylabel("p-value ($p_t$)")
plt.title("p-values for Gaussian mean change (learned score)")
plt.legend()
plt.savefig("images/learned.pdf")
plt.show()